# Testando o Modelo

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
for step in range(5):

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hello


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


DialoGPT: Hello! :D
>> User:how are you
DialoGPT: I'm good, how are you?
>> User:im fine
DialoGPT: That's good
>> User:bye
DialoGPT: Bye! :D
>> User::)
DialoGPT: Bye! :D


# Criando chatbot

In [5]:
import pandas as pd

dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}

df_status_pedidos = pd.DataFrame(dados_pedidos)

In [6]:
df_status_pedidos

,numero_pedido,status
0,12345,Shipped
1,67890,Processing
2,11121,Delivered
3,22232,Cancelled


In [7]:
def verificar_status_pedido(numero_pedido):
    try:
        status = df_status_pedidos[df_status_pedidos["numero_pedido"] == numero_pedido]["status"].iloc[0]
        return f"The status of your order {numero_pedido} is: {status}"
    except:
        return "Order number not found. Please check and try again."

In [8]:
palavras_chave_status = ["order", "order status", "status of my order", "check my order", "track my order", "order update"]

In [10]:
ids_historico_chat = None

while True:
    input_usuario = input('You: ')

    if input_usuario.lower() in ['exit', 'quit', 'stop']:
        print('Bot: Goodbye!')
        break

    if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
        numero_pedido = input('Could you please enter your order number?')
        resposta = verificar_status_pedido(numero_pedido)
    else:
        novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

        if ids_historico_chat is not None:
            bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)

        else:
            bot_input_ids = novo_usuario_input_ids

        ids_historico_chat = model.generate(
            bot_input_ids,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id
        )
        resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    print(f'Bot: {resposta}')

You: Hi
Bot: Hi! :D
You: i need to know my order status
Could you please enter your order number?12345
Bot: The status of your order 12345 is: Shipped
You: ok bye
Bot: Bye! :D
You: quit
Bot: Goodbye!


# Construindo interface

In [11]:
!pip install gradio

In [12]:
# Importações necessárias
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import gradio as gr

In [13]:
# Carrega o tokenizador e o modelo
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [17]:
# Criando um dataframe com status de pedidos
dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}
df_status_pedidos = pd.DataFrame(dados_pedidos)

In [18]:
# Função para verificar o status do pedido
def verificar_status_pedido(numero_pedido):
  try:
    status = df_status_pedidos[df_status_pedidos['numero_pedido'] == numero_pedido]['status'].iloc[0]

    return f'The status of your order {numero_pedido} is: {status}'

  except:
    return 'Order number not found. Please check and try again'

# Lista de palavras-chave para o status
palavras_chave_status = ['order', 'order status', 'status of my order', 'check my order', 'track my order', 'order update']

In [19]:
def responder(input_usuario, ids_historico_chat):
  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    return 'Could you please enter you order number?', ids_historico_chat
  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids ], dim=-1)

    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return resposta, ids_historico_chat

In [22]:
# Criando interface
with gr.Blocks() as app:
  chatbot = gr.Chatbot()
  msg = gr.Textbox(placeholder='Type your message here...')

  estado = gr.State(None)
  aguardando_numero_pedido = gr.State(False)

  def processar_entrada(input_usuario, historico, ids_historico_chat, aguardando_numero_pedido):
    if aguardando_numero_pedido:
      resposta = verificar_status_pedido(input_usuario)
      aguardando_numero_pedido = False
    else:
      resposta, ids_historico_chat = responder(input_usuario, ids_historico_chat)
      if resposta == 'Could you please enter you order number?':
        aguardando_numero_pedido = True

    historico.append((input_usuario, resposta))
    return historico, ids_historico_chat, aguardando_numero_pedido, ""


  msg.submit(
      processar_entrada,
      [msg, chatbot, estado, aguardando_numero_pedido],
      [chatbot, estado, aguardando_numero_pedido, msg]
  )

  app.launch(share=True)

/tmp/ipython-input-3409580414.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e00928838528ea5b4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
